In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

dtype = torch.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 3 words sentences (=sequence_length is 3)
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.
embedding_size = 2 #训练样本比较少，不需要太高维度
sequence_length = len(sentences[0]) # 如果句子不等长则需要padding
num_classes = len(set(labels))
batch_size = 3

word_list = " ".join(sentences).split()
vocab = list(set(word_list))
word2idx = {w: i for i, w in enumerate(vocab)}
vocab_size = len(vocab)

In [7]:
def make_data(sentences,labels):
    inputs = []
    for sen in sentences:
        inputs.append([word2idx[n] for n in sen.split()])
    return inputs, labels

input_data, output_data = make_data(sentences,labels)
input_data, output_data = torch.LongTensor(input_data), torch.LongTensor(output_data)
dataset = Data.TensorDataset(input_data, output_data)
loader = Data.DataLoader(dataset, batch_size, True)


In [10]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim= embedding_size)
        output_channels = 3
        self.conv = nn.Sequential(
            # conv : [input_channel(=1), output_channel, (filter_height, filter_width), stride=1]
            nn.Conv2d(1,output_channels,(2, embedding_size)), #每两个词做一个conv # => [batch_size, output_channel, 2, 1]
            nn.ReLU(),
            # pool : ((filter_height, filter_width))
            nn.MaxPool2d((2,1)),
        )
        self.fc = nn.Linear(output_channels, num_classes)
    def forward(self,x):
        '''
        X: [batch_size, sequence_length]
        '''
        batch_size = x.shape[0]
        embedding1 = self.embedding(x) # [batch_size, sequence_length, embedding_size]
        embedding1 = embedding1.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]
        conved = self.conv(embedding1) # [batch_size, output_channel, 1, 1]
        flatten = conved.view(batch_size,-1) # [batch_size, output_channel*1*1]
        out = self.fc(flatten)
        return out 

In [11]:
model = TextCNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training
for epoch in range(5000):
  for batch_x, batch_y in loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    pred = model(batch_x)
    loss = criterion(pred, batch_y)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 1000 loss = 0.069539
Epoch: 1000 loss = 0.001121
Epoch: 2000 loss = 0.020395
Epoch: 2000 loss = 0.000258
Epoch: 3000 loss = 0.007320
Epoch: 3000 loss = 0.000051
Epoch: 4000 loss = 0.002826
Epoch: 4000 loss = 0.000016
Epoch: 5000 loss = 0.000004
Epoch: 5000 loss = 0.001125


In [12]:
# Test
test_text = 'i hate me'
tests = [[word2idx[n] for n in test_text.split()]]
test_batch = torch.LongTensor(tests).to(device)
# Predict
model = model.eval()
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

i hate me is Bad Mean...


In [3]:
from collections import deque

def min_days_to_grow_m_grass(m, n, points):
    # 创建空的二维平面
    grid = [[0] * 1000000001 for _ in range(1000000001)]
    # 将所有草的初始位置标记为“已生长”
    for x, y in points:
        grid[x][y] = 1
    # BFS 遍历
    days = 0
    queue = deque(points)
    while queue:
        size = len(queue)
        for _ in range(size):
            x, y = queue.popleft()
            for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
                nx, ny = x + dx, y + dy
                if nx >= 0 and ny >= 0 and nx <= 1000000000 and ny <= 1000000000 and not grid[nx][ny]:
                    grid[nx][ny] = 1
                    queue.append((nx, ny))
        days += 1
        # 检查是否已经找到至少 M 种草
        for x, y in queue:
            count = 0
            for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
                nx, ny = x + dx, y + dy
                if nx >= 0 and ny >= 0 and nx <= 1000000000 and ny <= 1000000000 and grid[nx][ny]:
                    count += 1
            if count >= m:
                return days
    return 0


In [4]:
# 输入
M = 3
n = 5
points = [[0, 0], [1, 1], [1, 0], [0, 1], [2, 2]]

# 输出
# 所有草都在同一点，因此第0天就可以找到一点至少有3种草
print(find_point_with_m_grass(M, n, points))  # 输出: 0

# 修改第5朵草的位置
points[4] = [0, 2]

# 输出
# 第1天时，(0,1)这个点有3种草
print(find_point_with_m_grass(M, n, points))  # 输出: 1


NameError: name 'find_point_with_m_grass' is not defined

In [21]:
import heapq

class Solution:
    def __init__(self, M, n, points):
        self.M = M
        self.n = n
        self.points = points
        self.days = 0
        self.visited = set()
        self.adjacent = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        
    def is_valid(self, x, y):
        return 1 <= x <= 10**9 and 1 <= y <= 10**9
    
    def bfs(self, heap):
        while heap:
            curr = heapq.heappop(heap)
            x, y, types = curr
            if len(types) >= self.M:
                return self.days
            for dx, dy in self.adjacent:
                nx, ny = x + dx, y + dy
                if (nx, ny) not in self.visited and self.is_valid(nx, ny):
                    self.visited.add((nx, ny))
                    for p in self.points:
                        if p == [nx, ny] or p in types:
                            heapq.heappush(heap, (nx, ny, types | set([tuple(p)])))
        return 0
    
    def find_min_days(self):
        heap = []
        for p in self.points:
            heap.append((p[0], p[1], set([tuple(p)])))
            self.visited.add(tuple(p))
        heapq.heapify(heap)
        return self.bfs(heap)

In [22]:
m = 3
n = 5
points = [(1,1),(2,2),(3,1),(5,5),(5,6)]
result = Solution(m, n, points)

print(result.find_min_days())


KeyboardInterrupt: 

In [23]:
class ResourcePool:
    def __init__(self, n):
        self.pool = list(range(1, n+1))
        self.used = set()
        
    def allocate(self, x=None):
        if x is None:
            if self.pool:
                x = self.pool.pop(0)
                self.used.add(x)
        else:
            if x not in self.used and x in self.pool:
                self.pool.remove(x)
                self.used.add(x)
        return x
    
    def release(self, x):
        if x in self.used:
            self.used.remove(x)
            self.pool.append(x)
        return
    
    def get_first_free(self):
        if self.pool:
            return self.pool[0]
        else:
            return None


In [ ]:
n = int(input())  # 资源池的范围
m = int(input())  # 操作个数
rp = ResourcePool(n)

for i in range(m):
    op, x = map(int, input().split())
    if op == 1:
        x = rp.allocate()
        if x is not None:
            print(x)
        else:
            print("NULL")
    elif op == 2:
        rp.allocate(x)
    elif op == 3:
        rp.release(x)

print(rp.get_first_free())
